In [1]:
from gerrychain import Graph

In [2]:
# Read Oklahoma county graph from the json file "OK_county.json"
filepath = 'C:\\Users\sterl\OneDrive\Documents\\'
filename = 'COUNTY_08.json'

# GerryChain has a built-in function for reading graphs of this type:
G = Graph.from_json( filepath + filename )

In [3]:
# Let's impose a 2% population deviation (+/- 0.5%)
deviation = 0.2

import math
k = 7          # number of districts
total_population = sum(G.nodes[node]['TOTPOP'] for node in G.nodes)

L = math.ceil((1-deviation/2)*total_population/k)
U = math.floor((1+deviation/2)*total_population/k)
print("Using L =",L,"and U =",U,"and k =",k)

Using L = 646611 and U = 790302 and k = 7


In [4]:
import gurobipy as gp
from gurobipy import GRB

# create model 
m = gp.Model()

# create variables
x = m.addVars(G.nodes, k, vtype=GRB.BINARY) # x[i,j] equals one when county i is assigned to district j
y = m.addVars(G.edges, vtype=GRB.BINARY)  # y[u,v] equals one when edge {u,v} is cut

Set parameter Username


In [5]:
# objective is to minimize the total perimeter of the districts
#  the boundary length between counties u and v is stored in G.edges[u,v]['shared_perim']
m.setObjective( gp.quicksum( G.edges[u,v]['shared_perim']*y[u,v] for u,v in G.edges ), GRB.MINIMIZE )

In [6]:
# add constraints saying that each county i is assigned to one district
m.addConstrs( gp.quicksum(x[i,j] for j in range(k)) == 1 for i in G.nodes)

# add constraints saying that each district has population at least L and at most U
m.addConstrs( gp.quicksum( G.nodes[i]['TOTPOP'] * x[i,j] for i in G.nodes) >= L for j in range(k) )
m.addConstrs( gp.quicksum( G.nodes[i]['TOTPOP'] * x[i,j] for i in G.nodes) <= U for j in range(k) )

# add constraints saying that edge {i,j} is cut if i is assigned to district v but j is not.
m.addConstrs( x[i,v] - x[j,v] <= y[i,j] for i,j in G.edges for v in range(k))

m.update()

In [7]:
# Now, let's add contiguity constraints and re-solve the model.
# We will use the contiguity constraints of Hojny et al. (MPC, 2021)
#   https://link.springer.com/article/10.1007/s12532-020-00186-3

# Add root variables: r[i,j] equals 1 if node i is the "root" of district j
r = m.addVars(G.nodes, k, vtype=GRB.BINARY)

# Add flow variables: f[u,v] = amount of flow sent across arc uv 
#  Flows are sent across arcs of the directed version of G which we call DG
import networkx as nx
DG = nx.DiGraph(G) # directed version of G
f = m.addVars(DG.edges, vtype=GRB.CONTINUOUS)

In [8]:
# The big-M proposed by Hojny et al.
M = G.number_of_nodes() - k + 1

# Each district j should have one root
m.addConstrs( gp.quicksum( r[i,j] for i in DG.nodes) == 1 for j in range(k) )

# If node i is not assigned to district j, then it cannot be its root
m.addConstrs( r[i,j] <= x[i,j] for i in DG.nodes for j in range(k) )  

# if not a root, consume some flow.
# if a root, only send out (so much) flow.
m.addConstrs( gp.quicksum( f[u,v] - f[v,u] for u in DG.neighbors(v) ) >= 1 - M * gp.quicksum( r[v,j] for j in range(k)) for v in G.nodes)

# do not send flow across cut edges
m.addConstrs( f[i,j] + f[j,i] <= M * (1 - y[i,j]) for (i,j) in G.edges )

m.update()

In [ ]:
# solve IP model
m.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1885 rows, 1379 columns and 7644 nonzeros
Model fingerprint: 0x674952b9
Variable types: 322 continuous, 1057 integer (1057 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+05]
  Objective range  [1e-03, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+05]
Presolve time: 0.04s
Presolved: 1885 rows, 1379 columns, 7574 nonzeros
Variable types: 322 continuous, 1057 integer (1057 binary)

Root relaxation: objective 5.045794e+00, 918 iterations, 0.14 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.04579    0  507          -    5.04579      -     -    0s
     0     0    5.

 1360630 89472   14.04698   41  229   14.23463   13.60042  4.46%  86.6 2486s
 1364981 88551   13.80120   53  156   14.23463   13.60526  4.42%  86.6 2490s
 1369229 87669     cutoff   54        14.23463   13.60981  4.39%  86.5 2496s
 1373284 86687   13.83030   36  314   14.23463   13.61464  4.36%  86.5 2500s
 1378790 85400   13.93804   38  258   14.23463   13.62059  4.31%  86.4 2506s
 1382698 84432   13.94138   44  154   14.23463   13.62535  4.28%  86.4 2510s
 1386814 83337     cutoff   36        14.23463   13.63060  4.24%  86.3 2515s
 1391156 82216   14.20802   41  278   14.23463   13.63573  4.21%  86.3 2520s
H1394606 81381                      14.2346263   13.63952  4.18%  86.2 2523s
 1395496 81278     cutoff   49        14.23463   13.64045  4.17%  86.2 2526s
 1399202 80073   14.02076   46  271   14.23463   13.64526  4.14%  86.2 2530s
 1403833 78789     cutoff   51        14.23463   13.65077  4.10%  86.1 2535s
H1407178 77999                      14.2346263   13.65482  4.07%  86.1 2538s

In [ ]:
print("The total perimeter length (excluding the exterior boundary of state) is",m.objval)

# retrieve the districts and their populations
districts = [ [i for i in G.nodes if x[i,j].x > 0.5] for j in range(k)]
district_counties = [ [ G.nodes[i]["NAME10"] for i in districts[j] ] for j in range(k)]
district_populations = [ sum(G.nodes[i]["TOTPOP"] for i in districts[j]) for j in range(k) ]

# print district info
for j in range(k):
    print("District",j,"has population",district_populations[j],"and contains counties",district_counties[j])

In [ ]:
# Let's draw it on a map
import geopandas as gpd

In [ ]:
# Read Oklahoma county shapefile from "OK_county.shp"
filepath = 'C:\\Users\sterl\OneDrive\Documents\\'
filename = 'CO_counties1.shp'

# Read geopandas dataframe from file
df = gpd.read_file( filepath + filename )

In [ ]:
# Which district is each county assigned to?
assignment = [ -1 for u in G.nodes ]
    
# for each district j
for j in range(len(districts)):
    
    # for each node i in this district
    for i in districts[j]:
        
        # What is its GEOID?
        geoID = G.nodes[i]["GEOID10"]
        
        # Need to find this GEOID in the dataframe
        for u in G.nodes:
            if geoID == df['GEOID10'][u]: # Found it
                assignment[u] = j # Node u from the dataframe should be assigned to district j

# Now add the assignments to a column of the dataframe and map it
df['assignment'] = assignment
my_fig = df.plot(column='assignment').get_figure()